In [3]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2
from collections import defaultdict
import re
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd   
from gensim import corpora, models, similarities, matutils
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from gensim.matutils import cossim
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
import pickle
import random
import sys
from tqdm import tqdm
from sklearn import manifold

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
string = "Nunca pertenecería a un club que admitiera como socio a 1 persona como yo"

In [5]:
string

'Nunca pertenecería a un club que admitiera como socio a 1 persona como yo'

In [6]:
# Tokenizar
tokens = word_tokenize(string)
tokens

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  Searched in:
    - '/home/gastonb/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/home/gastonb/anaconda3/nltk_data'
    - '/home/gastonb/anaconda3/share/nltk_data'
    - '/home/gastonb/anaconda3/lib/nltk_data'
    - ''
**********************************************************************


In [ ]:
# largo de la lista
len(tokens)

In [ ]:
# muestro el 4to elemento de la lista
tokens[4]

In [ ]:
# muestro del 4to elemento de la lista en adelante
tokens[4:]

In [ ]:
# muestro hasta el 4to elemento de la lista
tokens[:4]

In [ ]:
# paso a uppercase e imprimo
for s in tokens:
    print( s.upper() )

In [ ]:
# si el token tiene una "a", lo paso a uppercase e imprimo
for s in tokens:
    if "a" in s:
        print( s.upper() )

In [ ]:
# No solo imprimo, sino que guardo las palabras en una lista
tokens_filtrados=[]
for s in tokens:
    if "a" in s:
        tokens_filtrados.append(s.upper())
        print( tokens_filtrados )

In [ ]:
# muestro la lista guardada
tokens_filtrados

### Ejemplo de limpieza  

In [ ]:
# el .decode("utf-8") le dice a python que este tecto tiene caracteres "latinos"
string = "Nunca pertenecería a un club que admitiera como socio a 1 persona como yo. No puedo decir que no estoy en desacuerdo contigo. Citadme diciendo que me han citado mal (Ni $10.5)"
string

In [ ]:
# remplazo los signos $
string_step1 = string.replace("$"," signopesos ")  
string_step1

In [ ]:
# reemplazo numeros con " num " 
reg_num = re.compile(r"\d+[.,]?\d*") # Regular expression to search numbers
string_step2 = reg_num.sub(" NUM ",string_step1)  
string_step2

In [ ]:
# paso a lowercase
string_step3 = string_step2.lower()

In [ ]:
word_tokenize(string_step3)

In [ ]:
# tiro los que no son alphabeticos 
tokens_cleaned = []
for token in word_tokenize(string_step3):
    if token.isalpha(): # acá resulta relevante si el texto esta en utf-8
        tokens_cleaned.append(token)

tokens_cleaned

## Separacion por oración

In [ ]:
string = "Nunca pertenecería a un club que admitiera como socio a 1 persona como yo. No puedo decir que no estoy en desacuerdo contigo. Citadme diciendo que me han citado mal (Ni $10.5)"
# remplazo los signos $
string_step1 = string.replace("$"," signopesos ") 
# reemplazo numeros con " num " y  paso a lowercase 
string_step3 = reg_num.sub(" NUM ",string_step1).lower()  
string_step4 = sent_tokenize(string_step3)

# tiro los tokens no alphabeticos 
string_cleaned = []
for sent in string_step4:
    tokens = []
    for token in word_tokenize(sent):
        if token.isalpha():
            tokens.append(token)
    string_cleaned.append(tokens)
    
string_cleaned

# Word2vec tutorial
## Corpus Cleaning

In [ ]:
corpus = open("diario_09-11.txt", "r").read() # read corpus
corpus[:1000] # mostrar primeras 1000 palabras


In [ ]:
sent_tokenize(corpus[:1000]) # armo una lista de oraciones

In [ ]:
############################ ir corriendo ##########
# remplazo los signos $
corpus = corpus.replace("$"," signopesos ") 
# reemplazo numeros con " num " y  paso a minuscula 
corpus = reg_num.sub(" NUM ",corpus).lower()  
corpus = sent_tokenize(corpus)
# tiro los tokens no alphabeticos 
trainset = []
for sent in tqdm(corpus):
    tokens = []
    for token in word_tokenize(sent):
        if token.isalpha():
            tokens.append(token)
    trainset.append(tokens)

In [ ]:
trainset[:2]

In [ ]:
print ("el corpus tiene",len(trainset), "oraciones y",sum([len(x) for x in trainset]),"palabras"   )

In [ ]:
# filtro oraciones cortas
trainset2 = [] 
for sent in trainset:
    if len(sent)>3:
        trainset2.append(sent)

print( "el corpus tiene",len(trainset2), "oraciones y",sum([len(x) for x in trainset2]),"palabras"  )

In [ ]:
# "window" es el tamaño de la ventana. windows = 10, usa 10 palabras a la izquierda y 10 palabras a la derecha
# "n_dim" es la dimension (i.e. el largo) de los vectores de word2vec
# "workers" es el numero de cores que usa en paralelo. Para aprobechar eso es necesario tener instalado Cython)
# "sample": word2vec filtra palabras que aparecen una fraccion mayor que "sample"
# "min_count": Word2vec filtra palabras con menos apariciones que  "min_count"
# "sg": para correr el Skipgram model (sg = 1), para correr el CBOW (sg = 0)
# para mas detalle ver: https://radimrehurek.com/gensim/models/word2vec.html
n_dim = 20
w2v_model = Word2Vec(trainset2, workers=4,size=n_dim, min_count = 10, window = 10, sample = 1e-3,negative=10,sg=1)
#w2v_model.save("word2vec_diario_20dim")  # save model
#w2v_model = Word2Vec.load("word2vec_tasa")  # load model

In [ ]:
w2v_model = Word2Vec.load("word2vec_diario_20dim")  # load model

In [ ]:
# the output of a word2vec representation is a numpy array 
w2v_model["arte"]

In [ ]:
print ("mujer-cocina similarity:",w2v_model.wv.n_similarity(["mujer"], ["cocina"]))
print ("hombre-cocina similarity:",w2v_model.wv.n_similarity(["hombre"], ["cocina"]) )
print ("\n")
print ("mujer-esposa similarity:",w2v_model.wv.n_similarity(["mujer"], ["esposa"]) )
print ("hombre-esposo similarity:",w2v_model.wv.n_similarity(["hombre"], ["esposo"]) )
print("\n")
print ("mujer-hijos similarity:",w2v_model.wv.n_similarity(["mujer"], ["hijos"]) )
print ("hombre-hijos similarity:",w2v_model.wv.n_similarity(["hombre"], ["hijos"]) )


In [ ]:
w2v_model.most_similar(positive=["biología"], negative=[], topn=25)

In [ ]:
w2v_model.most_similar(positive=["computación"], negative=[], topn=25)

In [ ]:
target_word="crimen"
barrios = ["belgrano","caballito","ortúzar","palermo","recoleta","núñez","lugano","pompeya","martelli","flores","barracas","soldati","cañitas"]
crimen = []
for word in barrios:
    crimen.append(w2v_model.wv.n_similarity([target_word], [word]))
    
pd.DataFrame(crimen,index = barrios,columns=[target_word]).sort_values(by=target_word).plot(kind="bar")

In [ ]:
p_robos = ["robos","armas","asesinato","ladrones","hurto","asalto"]
p_ciencias = ["biología","química","matemática","filosofía","psicología","ciencia","ingeniería"]
p_tiempo = ["lluvioso","soleado","calor","nublado","nieve","tormenta"]
p_paises = ["suiza","suecia","francia","holanda","australia","perú","bolivia","paraguay","uruguay","brasil","colombia"]
colores = ["black"]*len(p_robos)+["blue"]*len(p_ciencias)+["green"]*len(p_tiempo)+["red"]*len(p_paises) 
grupos = [p_robos,p_ciencias,p_tiempo,p_paises]
palabras = p_robos + p_ciencias + p_tiempo + p_paises

In [ ]:
# Armo una matriz de distancias
distancias=np.zeros((len(palabras),len(palabras))) #matriz cuadrada
for i,ti in enumerate(palabras):
    for j,tj in enumerate(palabras):
        distancias[i,j] = abs(1-w2v_model.wv.similarity(ti,tj))
print (distancias.shape)
distancias

In [ ]:
# Reduccion de la dimensionalidad y visualizacion 
from sklearn.manifold import TSNE 
tsne = TSNE(n_components=2,metric="precomputed",learning_rate=1000, random_state=123)
np.set_printoptions(suppress=True)
plt.figure(figsize=(8,8))
Y = tsne.fit_transform(distancias)
plt.scatter(Y[:, 0], Y[:, 1],color="black",s=3)
for label, x, y, color in zip(palabras, Y[:, 0], Y[:, 1],colores):
    plt.annotate(label, xy=(x, y), xytext=(0, 0),color=color, textcoords='offset points')

# Latent Semantic Analisis (LSA)

In [ ]:
# Creates a dictionary wich maps tokens with Ids
dictionary = corpora.Dictionary(trainset2)

In [ ]:
list(dictionary.iteritems())[:10]

In [ ]:
# filter words with low frequency (less than 5) 
dictionary.filter_extremes(no_below=10, no_above=1, keep_n=100000)
list(dictionary.iteritems())[:10]

In [ ]:
stopwords.words('spanish')[:10]

In [ ]:
stopwords_id =np.array(dictionary.doc2bow(stopwords.words('spanish')))[:,0]
dictionary.filter_tokens(bad_ids=stopwords_id, good_ids=None)
dictionary.save("diarios_dictionary.dict")

In [ ]:
corpus = [dictionary.doc2bow(line) for line in trainset2]
tfidf = models.TfidfModel(corpus)  # tf-idf  transformation
corpus_tfidf = tfidf[corpus]

In [ ]:
n_topics = 100
lsa_tfidf = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics = n_topics)  # initialize an LSI transformation
#lsa_tfidf.save("LSA_diarios.lsi") # Save LSA

In [ ]:
# Load LSA
lsa_tfidf = models.LsiModel.load("LSA_diarios.lsi")

In [ ]:
trainset2[1111]

In [ ]:
corpus[1111] # (id,frequency) of the tokens in document 123

In [ ]:
print ("id num ->", dictionary.doc2bow(["num"]),"(2 rep)") # id -> token
print ("id noches ->", dictionary.doc2bow(["noches"]),"(1 rep)") # id -> token
print ("id mismo ->", dictionary.doc2bow(["mismo"]),"(1 rep)") # id -> token

In [ ]:
ids = [dictionary.doc2bow(["num"])[0][0],dictionary.doc2bow(["noches"])[0][0],dictionary.doc2bow(["mismo"])[0][0]]
pd.Series(dict(corpus_tfidf[1111]))[ids]# (id,tf-idf weight) of the tokens in document 1111

In [ ]:
# vectorial representation of a token
vect_arte = lsa_tfidf[dictionary.doc2bow(["arte"])] 
vect_arte[:10] # shows only the first 10 elements


In [ ]:
# show the components of topic 2
lsa_tfidf.show_topic(19, topn=19)

In [ ]:
# Armo una matriz de distancias
distancias_lsa=np.zeros((len(palabras),len(palabras))) #matriz cuadrada
for i,ti in enumerate(palabras):
    for j,tj in enumerate(palabras):
        distancias_lsa[i,j] = abs(1-cossim(lsa_tfidf[dictionary.doc2bow([ti])] ,lsa_tfidf[dictionary.doc2bow([tj])]))
print( distancias_lsa.shape )
distancias_lsa

In [ ]:
# Reduccion de la dimensionalidad y visualizacion 
from sklearn.manifold import TSNE 
tsne = TSNE(n_components=2,metric="precomputed",learning_rate=1000, random_state=123)
np.set_printoptions(suppress=True)
plt.figure(figsize=(8,8))
Y = tsne.fit_transform(distancias_lsa)
plt.scatter(Y[:, 0], Y[:, 1],color="black",s=3)
for label, x, y, color in zip(palabras, Y[:, 0], Y[:, 1],colores):
    plt.annotate(label, xy=(x, y), xytext=(0, 0),color=color, textcoords='offset points')